In [1]:
import os
import os.path as op
import glob
import numpy as np
import multiprocessing
from nipype.interfaces import fsl

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['sim_4mm']
runs = map(str,range(1,4))

In [3]:
#get cortical areas from Yeo atlas
path = home_dir + '/data/fsaverage/label/'
# cortical_areas =  glob.glob(path + '*7Net*')
# cortical_areas = [x.split('/')[-1] for x in cortical_areas]

#get cortical areas from Michael's parcellation
cortical_areas =  glob.glob(path + '*yeo*')
cortical_areas = [x.split('/')[-1] for x in cortical_areas]
print cortical_areas

#get vmpfc roi
# cortical_areas =  glob.glob(path + '*vmpfc*')
# cortical_areas = [x.split('/')[-1] for x in cortical_areas]
# cortical_areas = [cortical_areas[0]]
# cortical_areas

['lh.yeo17_ifs_thresh0.25.label', 'rh.yeo17_ains.label', 'lh.yeo17_dmfc.label', 'rh.yeo17_ifs.label', 'rh.yeo17_fpc.label', 'lh.yeo17_ains.label', 'rh.yeo17_pifs.label', 'rh.yeo17_ifg.label', 'lh.yeo17_pifs.label', 'lh.yeo17_fpc.label', 'rh.yeo17_ips.label', 'lh.yeo17_amfg.label', 'rh.yeo17_aifs.label', 'lh.yeo17_ifs.label', 'lh.yeo17_ips.label', 'lh.yeo17_pmfg.label', 'rh.yeo17_amfg.label', 'rh.yeo17_otc.label', 'lh.yeo17_psfs.label', 'lh.yeo17_otc.label', 'rh.yeo17_ifs_thresh0.25.label', 'lh.yeo17_aifs.label', 'lh.yeo17_ifg.label', 'rh.yeo17_psfs.label', 'rh.yeo17_dmfc.label', 'rh.yeo17_pmfg.label']


In [4]:
def make_mask(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    hemi = roi.split('.')[0]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           hemi + '.' + roi_id,
            '-label',
           roi_id,
           '-sample graymid',
#            '-proj frac 0 1 .1',
           '-hemi',
           hemi,
           '-debug',
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)

In [13]:
def make_mask_no_hemi(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           roi_id,
            '-label',
           roi_id,
           '-sample graymid',
           '-exp',exps[0],
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)

In [28]:
def make_contrast_masks(in_tuple):
    sub,exp,contrast = in_tuple
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           'caudate_' + contrast,
           '-aseg',
           '-id 11 50',
           '-exp',exp,
           '-contrast', contrast,
           '-thresh','1.7']
    cmd = ' '.join(cmd)
    os.system(cmd)
# make_contrast_masks(in_tuples[0])

In [29]:
in_tuples = []
exp = 'sim_4mm-PEdiffone'
contrast = 'PE_mf'
for sub in subs:
    in_tuples.append((sub,exp,contrast))

In [30]:
pool = multiprocessing.Pool(40)
pool.map(make_contrast_masks,in_tuples)
pool.terminate()
pool.join()

In [13]:
# in_tuples = []
# for sub in subs:
#     for roi in cortical_areas:
#         in_tuples.append((sub,roi))

In [14]:
# pool = multiprocessing.Pool(30)
# pool.map(make_mask_no_hemi,in_tuples)
# pool.terminate()
# pool.join()

# Map MNI ROI back into EPI space

In [40]:
rois = ['striatum']
exp = 'sim_4mm'
#warp group statistical image back into subject space
for sub in subs: 
    if True:#sub != 'fd_104':
        #set paths
        mask_path = home_dir + '/data/' + sub
        out_dir = mask_path + '/masks/'

        #copy warpfile to make ANTs happy
        warp = mask_path + '/normalization/inverse_warpfield.nii.gz'
        warpfield = mask_path + '/normalization/InverseWarp.nii.gz'
        cmd = ['ln','-s',warp, warpfield]
        os.system(' '.join(cmd))

        for roi in rois:
            im =  home_dir + '/analysis/striatum/' + roi + '.nii.gz'

            #warp to subject space
            out_warp = out_dir + roi + '_anat_space.nii.gz'
            ref_file = fsl.Info.standard_image("avg152T1_brain.nii.gz")
            affine = home_dir + '/data/' + sub + '/normalization/affine.txt'
            anat = home_dir + '/data/' + sub + '/anat/T1_native.nii.gz'
            if os.path.exists(out_warp):
                os.remove(out_warp)
            cmd_str = ['WarpImageMultiTransform','3',im,out_warp,'-i',affine,warpfield,
                       '-R',anat]
            cmd_str = ' '.join(cmd_str)
            os.system(cmd_str)

            #register to functional space
            sub_path = home_dir + 'analysis/' + exp + '/' + sub

            #register to func
            transform = sub_path + '/preproc/run_1/func2anat_tkreg.dat'
            mean_func = sub_path + '/preproc/run_1/mean_func.nii.gz'
            out_rigid = out_dir + exp + '_' + roi + '_func_space.nii.gz'
            if os.path.exists(out_rigid):
                os.remove(out_rigid)
            cmd_line = ['mri_vol2vol','--mov',mean_func,'--targ',out_warp,'--reg',transform,'--o',out_rigid,
                        '--no-save-reg','--inv']
            os.system(' '.join(cmd_line))

            #binarize
            out_mask = out_dir + exp + '_' + roi + '_mask.nii.gz'
            if os.path.exists(out_mask):
                os.remove(out_mask)
            cmd = ['fslmaths',out_rigid,'-thr','.1','-bin',out_mask]
            os.system(' '.join(cmd))


In [37]:
for sub in subs:
    anat = '/home/iballard/mag/analysis/workingdir/antswarp/_subject_id_' + sub + '/convertbrain/brain_out.nii.gz'
    new = '/home/iballard/mag/data/' + sub + '/anat/brain_out.nii.gz'
    
    os.rename(anat,new)

OSError: [Errno 2] No such file or directory

In [82]:
for sub in subs:
    roi_path = '/home/iballard/mag/data/' + sub + '/masks/'
    
    cmd = ['fslmaths',roi_path + 'lh.vmpfc_bartre','-add',
           roi_path + 'rh.vmpfc_bartre', roi_path + 'temp']
    os.system(' '.join(cmd))
    
    cmd = ['fslmaths',roi_path + 'temp','-bin',
           roi_path + 'vmpfc_freesurfer']
    os.system(' '.join(cmd))
    

# Split HPC into left and right

In [27]:
import nibabel as nib
for sub in subs:
    roi_path = '/home/iballard/fd/data/' + sub + '/masks/hipp.nii.gz'
    x_dim = nib.load(roi_path).shape[0]
    
    cmd = ['fslmaths', roi_path, '-bin',
           '-roi', str(int(x_dim/2)),'-1',
            '-1', '-1', '-1','-1','-1','-1', 
           roi_path[:-7] + '_left.nii.gz']
    os.system(' '.join(cmd))
    
    cmd = ['fslmaths', roi_path, '-bin',
           '-roi', '0',str(int(x_dim/2)),
           '-1', '-1', '-1','-1','-1','-1', 
           roi_path[:-7] + '_right.nii.gz']
    os.system(' '.join(cmd))

    